# Bibliotécas

In [ ]:
!pip install paho-mqtt==1.5.1
import paho.mqtt.client as mqtt
import json
import csv
import time
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn.model_selection import train_test_split

import tensorflow as tf
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM, GRU, SimpleRNN, Reshape, InputLayer

import joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.5.1-py3-none-any.whl size=61549 sha256=52dd371aa257b3f1e597faacd258cdb91f23b001974461fc47e4beef3ce38c28
  Stored in directory: /root/.cache/pip/wheels/b6/72/f9/44ca415a44f9cc7468fe4c59e6282938e504fadec09641fa63
Successfully built paho-mqtt


# Definições

In [ ]:
window = 42
df = pd.DataFrame(columns=['pwm', 'amb_t' , 'mos_t'])

# Modelos

In [ ]:
X_scaler = joblib.load('X_scaler.joblib')
y_scaler = joblib.load('y_scaler.joblib')
m = tf.keras.models.load_model("model.h5")
print ("Model loaded")

Model loaded


In [ ]:
def prever(X_scaled):
  return y_scaler.inverse_transform(m.predict(X_scaled))[0][0]

In [ ]:
# Using predicted values to predict next step
def forecast(feature, steps=20):
  xin =[]
  y_next = []
  X_pred = np.concatenate((feature,feature[:,-steps:,:]), axis=1)
  #print(X_pred.shape)
  fim = window+steps
  for i in range(window,fim):
    #print(i)
    xin = X_pred[:,i-window:i,:]#.reshape((1, window, X_pred.shape[2]))
    #print(xin.shape)
    X_pred[0, i,-1] = m.predict(xin, verbose=0)[0][0]

  return y_scaler.inverse_transform(X_pred[-1,-steps:,-1].reshape(-1, 1))


# Dados

In [ ]:
def data_2_feature(data):
  data = data.dropna(axis=0)
  data_head_1 = data.head(1).copy()
  while data.shape[0] < window:
    data = pd.concat([data_head_1, data], ignore_index=True)
  data = data.astype(float)[-window:]
  X = data.dropna(axis=1).drop(['mos_t'], axis = 1).to_numpy()
  #print(X.shape)
  y = data['mos_t'].to_numpy()
  X_scaled = X_scaler.transform(X)
  y_scaled = y_scaler.transform(y.reshape(y.shape[0], 1))
  X_scaled = np.append(X_scaled, y_scaled, axis=1)

  #if X_scaled.shape[0] < window:
  #  X_scaled = np.concatenate((np.repeat(X_scaled[1,:].reshape(1, 2), window - X_scaled.shape[0], axis=0), X_scaled), axis=0)
  #  y_scaled = np.concatenate((np.repeat(y_scaled[1].reshape(1, 1), window - y_scaled.shape[0], axis=0), y_scaled), axis=0)

  X_scaled = X_scaled.reshape(1, X_scaled.shape[0], X_scaled.shape[1])
  y_scaled = y_scaled.reshape(1, y_scaled.shape[0])
  return X_scaled #, y_scaled

# MQTT

In [ ]:
# Inicializa o cliente MQTT
client = mqtt.Client(client_id="csv_publisher")
mqtt_topic = "esp32"
time_old = ''

previsão = 0

def connect_mqtt(mqtt_topic = "esp32"):

  # Configurações de conexão MQTT
  mqtt_broker = "mqtt.tago.io"
  mqtt_port = 1883
  mqtt_username = "Token #2"
  #mqtt_password = "99b46d4a-edc6-41ec-8479-6ac89fc8159c" #device 4
  mqtt_password = "25bb401f-3f50-4665-97d7-ab87fc4fdcea" #device 3
  #mqtt_topic = "Temperatura"


    # Evento chamado quando a conexão MQTT é estabelecida
  def on_connect(client, userdata, flags, rc):
    print("Conectado ao broker MQTT com código de resultado: " + str(rc))

  def on_publish(client, userdata, result):
    df.to_csv('dados.csv', index=False)
    print("Message published")

  def on_message(client, userdata, message):
    global previsão
    # Example string
    json_string = str(message.payload)

    print("Received message '" + json_string + "' on topic '"
      + message.topic + "' with QoS " + str(message.qos))

    # Remove the leading 'b\'' and trailing '\'' characters
    #json_string = json_string[2:-1]

    # Parse the JSON string into a dictionary
    data = json.loads(message.payload.decode("utf-8"))

    # Extract the variable, value, and time
    variable = data["variable"]
    value = data["value"]
    time = data["time"]

    # store data
    df.loc[time, variable] = float(value)
    if df.loc[time, 'pwm'] > 0 and df.loc[time, 'amb_t'] > 0 and df.loc[time, 'mos_t'] > 0:
      feature = data_2_feature(df)

      try:
        erro = 100*(df.loc[time, 'mos_t'] - previsão)/df.loc[time, 'mos_t']
        send_value(value=erro, variable = "erro", unit= "%",
                tempo=time, mqtt_topic="predicao")
      except:
        pass

      previsão = prever(feature)
      send_value(value=previsão, variable = "predic", unit= "\u00B0C",
                tempo=time, mqtt_topic="predicao")

      foresc = forecast(feature=feature, steps=20)

      send_value(value=foresc[9][0], variable = "foresc_10", unit= "\u00B0C",
                tempo=time, mqtt_topic="predicao")

      send_value(value=foresc[19][0], variable = "foresc_20", unit= "\u00B0C",
                tempo=time, mqtt_topic="predicao")

    #df_in.loc[len(df_in)] = [variable, value, time]
    #df = df_in.pivot(index='Time', columns='Variable', values='Value')
    #df.index = pd.to_datetime(df.index)
    #df = df.reset_index()
    #df_in.to_csv('dados_in.csv', index=False)

    # Print the extracted values
    #print("Variable:", variable)
    #print("Value:", value)
    #print("Time:", time)

  # Define o callback de conexão
  client.on_connect = on_connect
  client.on_publish = on_publish
  client.on_message = on_message

  # Configure o usuário e senha do MQTT
  client.username_pw_set(mqtt_username, mqtt_password)

  # Conecta-se ao broker MQTT
  client.connect(mqtt_broker, mqtt_port, 60)
  client.subscribe(mqtt_topic)

In [ ]:
def send_value(value=0.0, variable = "predic", unit = "",tempo="", mqtt_topic="predicao"):

  #tempo = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

  payload = {}
  payload["variable"] = variable
  payload["value"] = "{:.2f}".format(value)
  payload["unit"] = unit
  payload["time"] = tempo
  # Publica o payload no tópico MQTT
  client.publish(mqtt_topic, json.dumps(payload))
  print("Dados publicados no tópico MQTT {0}: {1}".format(mqtt_topic, payload))

# teste

In [ ]:
connect_mqtt()
try:
    client.loop_forever()
except KeyboardInterrupt:
    pass

client.disconnect()
client.loop_stop()
df.to_csv('dados.csv', index=False)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 37ms/step
Dados publicados no tópico MQTT predicao: {'variable': 'predic', 'value': '68.20', 'unit': '°C', 'time': '2024-06-12 00:35:45'}
Dados publicados no tópico MQTT predicao: {'variable': 'foresc_10', 'value': '65.44', 'unit': '°C', 'time': '2024-06-12 00:35:45'}
Dados publicados no tópico MQTT predicao: {'variable': 'foresc_20', 'value': '63.05', 'unit': '°C', 'time': '2024-06-12 00:35:45'}
Message published
Message published
Message published
Message published
Received message 'b'{"variable":"pwm","value":127,"time":"2024-06-12 00:35:51","unit":""}'' on topic 'esp32' with QoS 0
Received message 'b'{"variable":"amb_t","value":23.9375,"time":"2024-06-12 00:35:51","unit":"\xc2\xb0C"}'' on topic 'esp32' with QoS 0
Received message 'b'{"variable":"mos_t","value":69.3125,"time":"2024-06-12 00:35:51","unit":"\xc2\xb0C"}'' on topic 'esp32' with QoS 0
Dados publicados no tópico MQTT predicao: {'v

In [ ]:
client.disconnect()
client.loop_stop()

3